In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import (
    LabelEncoder, OneHotEncoder, MinMaxScaler
    )
from sklearn.metrics import (
    precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, accuracy_score
    )

%load_ext autoreload
%autoreload 2

## Preprocessing

In [2]:
quake_frame = pd.read_csv('data/consolidated_data.csv')

quake_frame.drop(['id', 'Unnamed: 0', 'place', 'time', 'updated'], inplace=True, axis=1)

In [3]:
quake_frame.type.unique()

array(['sonic boom', 'earthquake', 'quarry blast', 'explosion',
       'nuclear explosion', 'mine collapse', 'other event',
       'chemical explosion', 'rock burst', 'ice quake', 'rockslide',
       'Rock Slide', 'landslide', 'quarry', 'building collapse',
       'mining explosion', 'sonicboom', 'acoustic noise', 'not reported',
       'experimental explosion', 'collapse', 'meteorite',
       'induced or triggered event', 'volcanic eruption', 'eq'],
      dtype=object)

In [4]:
natural_quakes = ['earthquake']
quake_frame['simple_label'] = quake_frame['type'].apply(lambda x: x not in natural_quakes)

In [5]:
sum(quake_frame['simple_label'])/len(quake_frame.index)

0.027042808333236575

## Simple model, no imputation

We'll start this off with a simple model, just a Random Forest for two classes that takes only rows that have no nans in them. Let's see how many we get.  
Then we'll split the data 80/20 and run training.

In [6]:
quake_frame.dropna(inplace=True)
quake_frame.isna().sum()

latitude           0
longitude          0
depth              0
mag                0
magType            0
nst                0
gap                0
dmin               0
rms                0
net                0
type               0
horizontalError    0
depthError         0
magError           0
magNst             0
status             0
locationSource     0
magSource          0
simple_label       0
dtype: int64

In [7]:
len(quake_frame)

1227408

In [8]:
sum(quake_frame['simple_label'])/len(quake_frame.index)

0.03523522740604591

In [9]:
quake_frame.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
latitude,1227408.0,37.361674,4.841731,0.000,35.964167,37.573000,38.817000,62.030667
longitude,1227408.0,-119.557707,10.027502,-179.098,-122.701333,-120.558833,-118.150167,179.661500
depth,1227408.0,6.016756,7.922880,-3.882,1.816000,4.413000,7.830000,211.000000
mag,1227408.0,1.258097,0.694405,-2.500,0.800000,1.180000,1.670000,5.840000
nst,1227408.0,17.010182,13.671235,0.000,8.000000,13.000000,22.000000,276.000000
gap,1227408.0,121.032150,65.767724,0.000,72.000000,105.000000,153.000000,360.000000
dmin,1227408.0,0.078264,0.342578,0.000,0.017120,0.037840,0.079990,141.160000
rms,1227408.0,0.097118,0.195847,0.000,0.030000,0.060000,0.130000,64.290000
horizontalError,1227408.0,0.801039,2.296862,0.000,0.270000,0.410000,0.720000,194.584100
depthError,1227408.0,2.773763,6.903563,0.000,0.490000,0.770000,1.460000,725.300000


Alright, this changes the proportions slightly, but not too bad. If anything, one might suggest that at least the mild increase in proportion of non-earthquakes offsets the reduced dataset a little.  
Okay, so the problematic values are no longer there, that's something.  
Let's try this.  
We'll start by mixing up the data frame, then encoding all the categories numerically and splitting it sklearn style.

In [10]:
quake_frame = quake_frame.sample(frac=1, random_state=42).reset_index(drop=True)

le = LabelEncoder()

cat_columns = ['magType', 'net', 'status', 'locationSource', 'magSource']

for cat in cat_columns:
    quake_frame = pd.concat([quake_frame,
                             pd.get_dummies(quake_frame[cat], prefix=cat)],
                            axis=1)

scale_cols = ['latitude', 'longitude', 'depth', 'mag', 'nst', 'gap', 'dmin', 'rms', 'horizontalError',
 'depthError', 'magError', 'magNst']

scaler = MinMaxScaler()

quake_frame[scale_cols] = scaler.fit_transform(quake_frame[scale_cols])

x_cols = ['latitude', 'longitude', 'depth', 'mag', 'nst', 'gap', 'dmin', 'rms', 'horizontalError', 'depthError',
 'magError', 'magNst', 'magType_Mb', 'magType_Md', 'magType_Ml', 'magType_Unknown', 'magType_ma', 'magType_mb',
 'magType_mc', 'magType_md', 'magType_me', 'magType_mh', 'magType_ml', 'magType_mlg', 'magType_mlr', 'magType_mw',
 'net_av', 'net_ci', 'net_hv', 'net_ismpkansas', 'net_ld', 'net_mb', 'net_nc', 'net_nm', 'net_nn', 'net_pr',
 'net_se', 'net_uu', 'net_uw', 'status_automatic', 'status_manual', 'status_reviewed', 'locationSource_av',
 'locationSource_ci', 'locationSource_hv', 'locationSource_ismp', 'locationSource_ld', 'locationSource_mb',
 'locationSource_nc', 'locationSource_nm', 'locationSource_nn', 'locationSource_pr', 'locationSource_se',
 'locationSource_uu', 'locationSource_uw', 'magSource_av', 'magSource_ci', 'magSource_hv', 'magSource_ismp',
 'magSource_ld', 'magSource_mb', 'magSource_nc', 'magSource_nm', 'magSource_nn', 'magSource_pr', 'magSource_se',
 'magSource_uu', 'magSource_uw']

y_col = ['simple_label']

In [11]:
x_cols + y_col

['latitude',
 'longitude',
 'depth',
 'mag',
 'nst',
 'gap',
 'dmin',
 'rms',
 'horizontalError',
 'depthError',
 'magError',
 'magNst',
 'magType_Mb',
 'magType_Md',
 'magType_Ml',
 'magType_Unknown',
 'magType_ma',
 'magType_mb',
 'magType_mc',
 'magType_md',
 'magType_me',
 'magType_mh',
 'magType_ml',
 'magType_mlg',
 'magType_mlr',
 'magType_mw',
 'net_av',
 'net_ci',
 'net_hv',
 'net_ismpkansas',
 'net_ld',
 'net_mb',
 'net_nc',
 'net_nm',
 'net_nn',
 'net_pr',
 'net_se',
 'net_uu',
 'net_uw',
 'status_automatic',
 'status_manual',
 'status_reviewed',
 'locationSource_av',
 'locationSource_ci',
 'locationSource_hv',
 'locationSource_ismp',
 'locationSource_ld',
 'locationSource_mb',
 'locationSource_nc',
 'locationSource_nm',
 'locationSource_nn',
 'locationSource_pr',
 'locationSource_se',
 'locationSource_uu',
 'locationSource_uw',
 'magSource_av',
 'magSource_ci',
 'magSource_hv',
 'magSource_ismp',
 'magSource_ld',
 'magSource_mb',
 'magSource_nc',
 'magSource_nm',
 'magSourc

In [12]:
sum(quake_frame['simple_label'])

43248

## Try oversampling with GAN

Using the **ND Dial: Imbalanced Algorithms** repository at https://github.com/dialnd/imbalanced-algorithms

In [13]:
from imbalanced_algorithms.gan import GAN

In [14]:
# required to re-train if necessary
# import tensorflow as tf
# gan.close()
# tf.reset_default_graph()

In [15]:
gan = GAN(num_epochs=200,
          batch_size=3000,
          d_hidden_dim=(768, 512, 384, 256),
          g_hidden_dim=(1024, 768, 512, 384, 256, 128),
          n_input=len(x_cols + y_col),  # Number of columns in quake_frame
          stddev=0.15,  # standard deviation for initialization noise
          d_learning_rate=0.001,
          g_learning_rate=0.00075,
          pretrain=False,
          random_state=42)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








In [16]:
gan.fit(quake_frame.loc[quake_frame['simple_label'] == True, x_cols + y_col].values, display_step=1)

Epoch: 1 loss_d_real: 0.0062 loss_d_fake: 0.0039 loss_g: 25.2442
Epoch: 2 loss_d_real: 0.0423 loss_d_fake: 0.0309 loss_g: 16.1519
Epoch: 3 loss_d_real: 0.8026 loss_d_fake: 0.1224 loss_g: 7.9913
Epoch: 4 loss_d_real: 0.0121 loss_d_fake: 0.1841 loss_g: 3.0552
Epoch: 5 loss_d_real: 0.0025 loss_d_fake: 0.0237 loss_g: 3.9146
Epoch: 6 loss_d_real: 0.0114 loss_d_fake: 0.0094 loss_g: 5.0364
Epoch: 7 loss_d_real: 0.0037 loss_d_fake: 0.0073 loss_g: 5.5310
Epoch: 8 loss_d_real: 0.0052 loss_d_fake: 0.0086 loss_g: 5.8310
Epoch: 9 loss_d_real: 0.0024 loss_d_fake: 0.0065 loss_g: 6.0723
Epoch: 10 loss_d_real: 0.0106 loss_d_fake: 0.0070 loss_g: 6.1712
Epoch: 11 loss_d_real: 0.0047 loss_d_fake: 0.0046 loss_g: 6.2245
Epoch: 12 loss_d_real: 0.0056 loss_d_fake: 0.0092 loss_g: 6.1081
Epoch: 13 loss_d_real: 0.0084 loss_d_fake: 0.0058 loss_g: 6.3621
Epoch: 14 loss_d_real: 0.0079 loss_d_fake: 0.0046 loss_g: 6.5224
Epoch: 15 loss_d_real: 0.0032 loss_d_fake: 0.0793 loss_g: 3.9845
Epoch: 16 loss_d_real: 0.0108 lo

In [17]:
synth_samples = pd.DataFrame(gan.sample(round((len(quake_frame.index) - sum(quake_frame['simple_label']))/2)), columns = x_cols + y_col)

We'll have to help out a little and round the one-hot-encoded values to make them properly encoded.

In [18]:
one_hot_cols = ['magType_Mb',
 'magType_Md',
 'magType_Ml',
 'magType_Unknown',
 'magType_ma',
 'magType_mb',
 'magType_mc',
 'magType_md',
 'magType_me',
 'magType_mh',
 'magType_ml',
 'magType_mlg',
 'magType_mlr',
 'magType_mw',
 'net_av',
 'net_ci',
 'net_hv',
 'net_ismpkansas',
 'net_ld',
 'net_mb',
 'net_nc',
 'net_nm',
 'net_nn',
 'net_pr',
 'net_se',
 'net_uu',
 'net_uw',
 'status_automatic',
 'status_manual',
 'status_reviewed',
 'locationSource_av',
 'locationSource_ci',
 'locationSource_hv',
 'locationSource_ismp',
 'locationSource_ld',
 'locationSource_mb',
 'locationSource_nc',
 'locationSource_nm',
 'locationSource_nn',
 'locationSource_pr',
 'locationSource_se',
 'locationSource_uu',
 'locationSource_uw',
 'magSource_av',
 'magSource_ci',
 'magSource_hv',
 'magSource_ismp',
 'magSource_ld',
 'magSource_mb',
 'magSource_nc',
 'magSource_nm',
 'magSource_nn',
 'magSource_pr',
 'magSource_se',
 'magSource_uu',
 'magSource_uw',
 'simple_label']

In [19]:
synth_samples[one_hot_cols] = synth_samples[one_hot_cols].apply(round).astype('int')

In [20]:
synth_samples.head()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,...,magSource_ld,magSource_mb,magSource_nc,magSource_nm,magSource_nn,magSource_pr,magSource_se,magSource_uu,magSource_uw,simple_label
0,0.635341,0.173858,0.023625,0.523611,0.070972,0.196466,0.0,0.029123,0.000000,0.080656,...,0,0,0,0,0,0,0,0,0,1
1,0.627177,0.165113,0.013989,0.534554,0.056572,0.317007,0.0,0.033753,0.000000,0.086150,...,0,0,0,0,0,0,0,0,0,1
2,0.647575,0.179874,0.000000,0.564497,0.000000,0.300294,0.0,0.000000,0.011607,0.004658,...,0,0,1,0,0,0,0,0,0,1
3,0.644466,0.178558,0.000000,0.566991,0.000000,0.281617,0.0,0.000000,0.002728,0.008632,...,0,0,1,0,0,0,0,0,0,1
4,0.630248,0.172529,0.025024,0.523802,0.078192,0.180035,0.0,0.024942,0.000000,0.075458,...,0,0,0,0,0,0,0,0,0,1


In [21]:
train_length = int(np.round(len(quake_frame.index) * 0.8))

In [22]:
train_X = pd.concat([quake_frame.loc[:train_length, x_cols], synth_samples[x_cols]])
train_y = pd.concat([quake_frame.loc[:train_length, y_col], synth_samples[y_col]])

valid_X = quake_frame.loc[train_length:, x_cols]
valid_y = quake_frame.loc[train_length:, y_col]

In [23]:
from hellinger_distance_criterion import HellingerDistanceCriterion

In [24]:
n_estim = 100
hdc = HellingerDistanceCriterion(1, np.array([2],dtype='int64'))

rfc = RandomForestClassifier(n_estimators=n_estim,
                             random_state=42)

In [25]:
rfc.fit(train_X, np.ravel(train_y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [26]:
preds = pd.DataFrame(rfc.predict(valid_X), columns=['predictions'])

In [27]:
prec = precision_score(valid_y, preds)
reca = recall_score(valid_y, preds)
roc = roc_auc_score(valid_y, preds)
f1 = f1_score(valid_y, preds)
acc = accuracy_score(valid_y, preds)
conf_mat = confusion_matrix(valid_y, preds)

print("Precision: ", prec)
print("Recall: ", reca)
print("ROC score: ", roc)
print("F1 score: ", f1)
print("Accuracy score: ", acc)

Precision:  0.9694278827857414
Recall:  0.877382319173364
ROC score:  0.9381822311660918
F1 score:  0.9211113119990357
Accuracy score:  0.994667633472108
